In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import json
import pandas as pd
from pathlib import Path
from copy import deepcopy
from functools import partial

from bellem.qa.ablation import answer_question, answer_question_cot, answer_question_cte, answer_question_cte_cot
from bellem.utils import set_seed, jprint
from bellem.musique.multihop import benchmark

set_seed(89)

/home/pc/anaconda3/envs/baris/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
from tqdm.auto import tqdm
tqdm.pandas()

In [4]:
# from datetime import datetime
# import random
# random.seed(datetime.now().timestamp())
# df = pd.read_json('../../data/generated/musique-evaluation/dataset.jsonl', orient='records', lines=True)
# ABLATION_RECORD_IDS = random.sample(df['id'].tolist(), 100)
# ABLATION_RECORD_IDS


In [5]:
from bellem.musique.constants import ABLATION_RECORD_IDS

df = pd.read_json('../../data/generated/musique-evaluation/dataset.jsonl', orient='records', lines=True)
df = df.set_index('id', drop=False).loc[ABLATION_RECORD_IDS].copy().reset_index(drop=True)
qd_df = pd.read_json('../../data/generated/musique-evaluation/question-decomposition.jsonl', orient='records', lines=True)
df = pd.merge(df.drop(columns=['question', 'question_decomposition']), qd_df, on='id', suffixes=('', ''))
# df = df.head(20)

print(df.shape)
df.head()

(100, 8)


,id,paragraphs,answer,answer_aliases,answerable,answers,question,question_decomposition
0,2hop__575188_342798,"[{'idx': 0, 'title': 'Liliana Mumy', 'paragrap...",Ahmad Shah Qajar,[Ahmad Shah Qajar],True,[Ahmad Shah Qajar],Who is the child of Mahmoud Mirza's father?,"[{'id': 575188, 'question': 'Who is Mahmoud Mi..."
1,2hop__731584_700117,"[{'idx': 0, 'title': 'KAPE', 'paragraph_text':...",Berrien County,[Berrien County],True,[Berrien County],In which county is the city to which KKVU is l...,"[{'id': 731584, 'question': 'To which city is ..."
2,2hop__690412_526810,"[{'idx': 0, 'title': 'Cabramatta Creek', 'para...",Chao Phraya River,[Chao Phraya River],True,[Chao Phraya River],For what river does the river on which Pa Sak ...,"[{'id': 690412, 'question': 'On which river is..."
3,2hop__263638_69048,"[{'idx': 0, 'title': 'Michael J. Barron', 'par...",Honorable Justice Abiodun Smith,[Honorable Justice Abiodun Smith],True,[Honorable Justice Abiodun Smith],Who is the Chief Judge of the Tebesa Nemine's ...,"[{'id': 263638, 'question': 'Where was Tebesa ..."
4,2hop__142842_68489,"[{'idx': 0, 'title': 'Perfect Night: Live in L...",Snapper Foster,[Snapper Foster],True,[Snapper Foster],Who did the performer of Night Rocker play on ...,"[{'id': 142842, 'question': 'Who performed Nig..."


In [6]:
perfect_retrieval_func = lambda docs, query: [doc for doc in docs if doc['is_supporting']]

In [7]:
N_RUNS = 3
results = []

In [8]:
for i in range(1, N_RUNS+1):
    _, scores = benchmark(df, answer_question, perfect_retrieval_func, ignore_errors=True)
    results.append({**scores, "retrieval": "groundtruth", "context": "paragraphs", "qa": "direct", "run": i})
    jprint(scores)

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.63,
  "f1": 0.7053517593517594,
  "fuzzy_match": 0.73
}


  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.61,
  "f1": 0.7033517593517593,
  "fuzzy_match": 0.73
}


  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.59,
  "f1": 0.6753517593517594,
  "fuzzy_match": 0.71
}


In [9]:
for i in range(1, N_RUNS+1):
    _, scores = benchmark(df, answer_question_cot, perfect_retrieval_func, ignore_errors=True)
    results.append({**scores, "retrieval": "groundtruth", "context": "paragraphs", "qa": "reasoning", "run": i})
    jprint(scores)

  0%|          | 0/100 [00:00<?, ?it/s]

Failed to answer the question 2hop__776377_857193
peer closed connection without sending complete message body (incomplete chunked read)
{
  "exact_match": 0.5,
  "f1": 0.602984126984127,
  "fuzzy_match": 0.6
}


  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.53,
  "f1": 0.6398253968253967,
  "fuzzy_match": 0.65
}


  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.47,
  "f1": 0.5995266955266956,
  "fuzzy_match": 0.61
}


In [10]:
for i in range(1, N_RUNS+1):
    _, scores = benchmark(df, answer_question_cte, perfect_retrieval_func, ignore_errors=True)
    results.append({**scores, "retrieval": "groundtruth", "context": "paragraphs", "qa": "triplets", "run": i})
    jprint(scores)

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.65,
  "f1": 0.7355527544351074,
  "fuzzy_match": 0.75
}


  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.64,
  "f1": 0.7221241830065359,
  "fuzzy_match": 0.73
}


  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.62,
  "f1": 0.7282551353874883,
  "fuzzy_match": 0.75
}


In [11]:
for i in range(1, N_RUNS+1):
    _, scores = benchmark(df, answer_question_cte_cot, perfect_retrieval_func, ignore_errors=True)
    results.append({**scores, "retrieval": "groundtruth", "context": "paragraphs", "qa": "reasoning+triplets", "run": i})
    jprint(scores)

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.62,
  "f1": 0.7161825396825396,
  "fuzzy_match": 0.74
}


  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.61,
  "f1": 0.718813492063492,
  "fuzzy_match": 0.74
}


  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.59,
  "f1": 0.6921545893719806,
  "fuzzy_match": 0.72
}


# Report

In [12]:
report_df = pd.DataFrame.from_records(results, columns=['context', 'retrieval', 'qa', 'run', 'exact_match', 'f1'])
report_df

,context,retrieval,qa,run,exact_match,f1
0,paragraphs,groundtruth,direct,1,0.63,0.705352
1,paragraphs,groundtruth,direct,2,0.61,0.703352
2,paragraphs,groundtruth,direct,3,0.59,0.675352
3,paragraphs,groundtruth,reasoning,1,0.50,0.602984
4,paragraphs,groundtruth,reasoning,2,0.53,0.639825
5,paragraphs,groundtruth,reasoning,3,0.47,0.599527
6,paragraphs,groundtruth,triplets,1,0.65,0.735553
7,paragraphs,groundtruth,triplets,2,0.64,0.722124
8,paragraphs,groundtruth,triplets,3,0.62,0.728255
9,paragraphs,groundtruth,reasoning+triplets,1,0.62,0.716183


In [13]:
report_df.drop(columns=['context', 'retrieval', 'run']).groupby(['qa']).mean()

,exact_match,f1
qa,,
direct,0.610000,0.694685
reasoning,0.500000,0.614112
reasoning+triplets,0.606667,0.709050
triplets,0.636667,0.728644
